In [2]:
import numpy as np

In [3]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

# API Key and URL
API_KEY = os.getenv("SPOONACULAR_API_KEY")
BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

# Parameters for the search
params = {
    "query": "pasta",  # Search term
    "includeIngredients": "tomato,cheese",  # Must-have ingredients
    "excludeIngredients": "nuts",  # Ingredients to exclude
    "diet": "vegetarian",  # Dietary preference
    "cuisine": "italian",  # Cuisine type
    "addRecipeInformation": True,  # Include detailed recipe info
    "number": 5,  # Number of results
    "apiKey": API_KEY  # API Key
}

# Make the API request
response = requests.get(BASE_URL, params=params)

# Check the response status and print results
if response.status_code == 200:
    recipes = response.json()
    print(recipes)
    for recipe in recipes.get("results", []):
        print(recipe)
        print(f"Recipe: {recipe['title']}")
        print(f"URL: {recipe.get('sourceUrl', 'N/A')}")
        print(f"Image: {recipe.get('image', 'N/A')}")
        print("-" * 30)
else:
    print(f"Failed to fetch recipes: {response.status_code}")
    print(response.json())


{'results': [{'vegetarian': True, 'vegan': False, 'glutenFree': False, 'dairyFree': False, 'veryHealthy': False, 'cheap': False, 'veryPopular': False, 'sustainable': False, 'lowFodmap': False, 'weightWatcherSmartPoints': 9, 'gaps': 'no', 'preparationMinutes': None, 'cookingMinutes': None, 'aggregateLikes': 1, 'healthScore': 24, 'creditsText': 'Foodista.com – The Cooking Encyclopedia Everyone Can Edit', 'license': 'CC BY 3.0', 'sourceName': 'Foodista', 'pricePerServing': 165.29, 'id': 644205, 'title': 'Garden Vegetable Ravioli With Tomato Brodo', 'readyInMinutes': 45, 'servings': 6, 'sourceUrl': 'https://www.foodista.com/recipe/Q8KRWHSR/garden-vegetable-ravioli-with-tomato-brodo', 'image': 'https://img.spoonacular.com/recipes/644205-312x231.jpg', 'imageType': 'jpg', 'summary': 'The recipe Garden Vegetable Ravioli With Tomato Brodo could satisfy your Mediterranean craving in approximately <b>45 minutes</b>. For <b>$1.65 per serving</b>, you get a hor d\'oeuvre that serves 6. One portion 

### To add subtitute for ingredients when asked for (API)


In [129]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

# API Key and URL
API_KEY = os.getenv("SPOONACULAR_API_KEY")
BASE_URL = "https://api.spoonacular.com/food/ingredients/substitutes"


# Parameters for the search
params = {
    "ingredientName": "pasta",
    "apiKey": API_KEY
}

# Make the API request
response = requests.get(BASE_URL, params=params)

# Check the response status and print results
if response.status_code == 200:
    substitutes = response.json()
    # print(substitutes)
    # print(f"{substitutes.get('message')}: {substitutes.get('ingredient')}")
    for substitute in substitutes.get("substitutes", []):
        print(substitute)
    #     print(f"Subtitutes: {recipe['substitutes']}")
    #     print(f"URL: {recipe.get('sourceUrl', 'N/A')}")
    #     print(f"Image: {recipe.get('image', 'N/A')}")
    #     print("-" * 30)
else:
    print(f"Failed to fetch recipes: {response.status_code}")
    print(response.json())

In [ ]:
from langchain_core.tools import tool

@tool
def search_ingredient_substitutes(ingredient_name: str) -> dict:
    """
    Fetches ingredient substitutes from the Spoonacular API.

    Args:
        ingredient_name (str): The name of the ingredient to find substitutes for.

    Returns:
        dict: A dictionary containing the substitutes and relevant information.
    """

    # Parameters for the request
    params = {
        "ingredientName": ingredient_name,
    }

    return params


def fetch_substitutes(parameter):
    # API Key and URL
    API_KEY = os.getenv("SPOONACULAR_API_KEY")
    BASE_URL = "https://api.spoonacular.com/food/ingredients/substitutes"


    # Parameters for the search
    params = {
        "ingredientName": parameter["ingredient_name"],
        "apiKey": API_KEY
    }

    # Make the API request
    response = requests.get(BASE_URL, params=params)

    # Check the response status and return results
    if response.status_code == 200:
        substitutes = response.json()
        return substitutes.get("substitutes", [])
    else:
        return {
            "error": f"Failed to fetch recipes: {response.status_code}",
            "details": response.json()
        }

In [ ]:
from langchain_core.tools import tool
import os
import requests
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env

@tool
def search_ingredient_substitutes(ingredient_name: str) -> dict:
    """
    Fetches ingredient substitutes from the Spoonacular API.

    Args:
        ingredient_name (str): The name of the ingredient to find substitutes for.

    Returns:
        dict: A dictionary containing the substitutes and relevant information.
    """
    # Return parameters for fetching substitutes
    return {
        "ingredient_name": ingredient_name,
        "api_key": os.getenv("SPOONACULAR_API_KEY")
    }


def fetch_substitutes(parameter: dict) -> dict:
    """
    Makes an API request to fetch substitutes for a given ingredient.

    Args:
        parameter (dict): A dictionary containing 'ingredient_name' and 'api_key'.

    Returns:
        dict: A dictionary containing the substitutes or error details.
    """
    # Validate API key
    if not parameter.get("api_key"):
        return {"error": "API key is missing. Please check your environment variables."}

    # API base URL
    BASE_URL = "https://api.spoonacular.com/food/ingredients/substitutes"

    # Parameters for the request
    params = {
        "ingredientName": parameter["ingredient_name"],
        "apiKey": parameter["api_key"]
    }

    try:
        # Make the API request
        response = requests.get(BASE_URL, params=params)

        # Validate response status
        if response.status_code == 200:
            substitutes = response.json()
            return {
                "substitutes": substitutes.get("substitutes", []),
                "message": substitutes.get("message"),
            }
        else:
            return {
                "error": f"Failed to fetch substitutes. Status code: {response.status_code}",
                "details": response.json(),
            }
    except requests.exceptions.RequestException as e:
        # Handle network-related errors
        return {"error": f"An error occurred while making the API request: {str(e)}"}


In [ ]:
from langchain_core.tools import tool

@tool
def search_recipes(
    query: str, 
    includeIngredients: str, 
    excludeIngredients: str, 
    diet: str, 
    cuisine: str, 
    addRecipeInformation: bool, 
    number: int, 
    apiKey: str
) -> dict:
    """Search for recipes based on various filters and preferences.

    Args:
        query: Search term for the recipe (e.g., 'pasta').
        includeIngredients: Ingredients that must be included in the recipes (comma-separated).
        excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
        diet: Dietary preference (e.g., 'vegetarian').
        cuisine: Cuisine type (e.g., 'italian').
        addRecipeInformation: Whether to include detailed recipe information (True/False).
        number: Number of results to return.
        apiKey: API key for authenticating the request.

    Returns:
        A dictionary containing the search results.
    """
    # Example implementation (replace with actual API call if needed)
    result = {
        "query": query,
        "filters": {
            "includeIngredients": includeIngredients.split(","),
            "excludeIngredients": excludeIngredients.split(","),
            "diet": diet,
            "cuisine": cuisine
        },
        "settings": {
            "addRecipeInformation": addRecipeInformation,
            "number": number
        },
        "apiKey": apiKey
    }
    return result


### The function for the tool calling

In [111]:
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
import os


def search_recipes_tool(query, includeIngredients, excludeIngredients, diet, cuisine, addRecipeInformation, number):

    # API Key and URL
    API_KEY = os.getenv("SPOONACULAR_API_KEY")
    BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

    # Parameters for the search
    params = {
        "query": query,  # Search term
        "includeIngredients": includeIngredients,  # Must-have ingredients
        "excludeIngredients": excludeIngredients,  # Ingredients to exclude
        "diet": diet,  # Dietary preference
        "cuisine": cuisine,  # Cuisine type
        "addRecipeInformation": addRecipeInformation,  # Include detailed recipe info
        "number": number,  # Number of results
        "apiKey": API_KEY  # API Key
    }

    # Make the API request
    response = requests.get(BASE_URL, params=params)

    # Check the response status and print results
    if response.status_code == 200:
        recipes = response.json()
        for recipe in recipes.get("results", []):
            return recipe
    else:
        return f"Failed to fetch recipes: {response.status_code}"



@tool
def search_recipes(
    query: str, 
    includeIngredients: str, 
    excludeIngredients: str, 
    diet: str, 
    cuisine: str, 
    addRecipeInformation: bool, 
    number: int,
) -> dict:
    """Search for recipes based on various filters and preferences.

    Args:
        query: Search term for the recipe (e.g., 'pasta').
        includeIngredients: Ingredients that must be included in the recipes (comma-separated).
        excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
        diet: Dietary preference (e.g., 'vegetarian').
        cuisine: Cuisine type (e.g., 'italian').
        addRecipeInformation: Whether to include detailed recipe information (True/False).
        number: Number of results to return.

    Returns:
        A dictionary containing the search results.
    """
    return search_recipes_tool(query, includeIngredients, excludeIngredients, diet, cuisine, addRecipeInformation, number)



In [112]:
from langchain_cohere import ChatCohere
import os

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")


llm = ChatCohere(model="command-r-plus-04-2024")

tools = [search_recipes]

In [113]:
llm_with_tools = llm.bind_tools(tools)

In [131]:
query = "I want a vegetarian recipe that includes mushrooms and spinach, excludes dairy and nuts, is high in protein, and takes less than 30 minutes to prepare. Provide 3 detailed results with nutritional information."

result = llm_with_tools.invoke(query).tool_calls
print(result)


[{'name': 'search_recipes', 'args': {'addRecipeInformation': 'True', 'cuisine': '', 'diet': 'vegetarian', 'excludeIngredients': 'dairy, nuts', 'includeIngredients': 'mushrooms, spinach', 'number': '3', 'query': 'vegetarian high protein'}, 'id': '2a2d5aa51556402782cb03b2b894d7db', 'type': 'tool_call'}]


In [119]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"search_recipes": search_recipes}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

{'vegetarian': False, 'vegan': False, 'glutenFree': True, 'dairyFree': False, 'veryHealthy': False, 'cheap': False, 'veryPopular': False, 'sustainable': False, 'lowFodmap': False, 'weightWatcherSmartPoints': 15, 'gaps': 'no', 'preparationMinutes': None, 'cookingMinutes': None, 'aggregateLikes': 2, 'healthScore': 36, 'creditsText': 'foodista.com', 'sourceName': 'foodista.com', 'pricePerServing': 206.4, 'id': 648155, 'title': 'Italian Kale and Potato Soup', 'readyInMinutes': 45, 'servings': 6, 'sourceUrl': 'https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup', 'image': 'https://img.spoonacular.com/recipes/648155-312x231.jpg', 'imageType': 'jpg', 'summary': 'Italian Kale and Potato Soup is a <b>gluten free and whole 30</b> recipe with 6 servings. This main course has <b>480 calories</b>, <b>22g of protein</b>, and <b>33g of fat</b> per serving. For <b>$2.06 per serving</b>, this recipe <b>covers 26%</b> of your daily requirements of vitamins and minerals. 2 people have m

[HumanMessage(content='Find a recipe I can make with chicken, onion, and garlic, that excludes peanuts and shellfish, is gluten-free, and provides 2 detailed results.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I will search for recipes that include chicken, onion and garlic, and exclude peanuts and shellfish, and are gluten-free. I will include two detailed results in my answer.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '3ae6bf7f-921e-4c5d-9e8c-157286257a85', 'tool_calls': [{'id': '3d4121086d9d433bab2820585d206ad2', 'function': {'name': 'search_recipes', 'arguments': '{"addRecipeInformation": "True", "cuisine": "", "diet": "gluten-free", "excludeIngredients": "peanuts, shellfish", "includeIngredients": "onion, garlic", "number": 2, "query": "chicken"}'}, 'type': 'function'}], 'token_count': {'input_tokens': 1108.0, 'output_tokens': 131.0}}, response_meta

In [118]:


import ast

# Provided ToolMessage content
tool_message_content = tool_message_contemt = messages[2].content

# Safely evaluate the string into a Python dictionary
tool_message_dict = ast.literal_eval(tool_message_content)

# Extract the sourceUrl from the dictionary
source_url = tool_message_dict.get('sourceUrl')

# Print the sourceUrl
if source_url:
    print(f"The source URL is: {source_url}")
else:
    print("No source URL found.")


The source URL is: https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup


In [103]:
source_url = messages.get('sourceUrl')

# Checking if the sourceUrl is available
if source_url:
    print(f"The recipe source URL is: {source_url}")
else:
    print("No source URL found in the ToolMessage.")

[HumanMessage(content='Find a recipe I can make with chicken, onion, and garlic, that excludes peanuts and shellfish, is gluten-free, and provides 2 detailed results.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I will search for recipes that include chicken, onion and garlic, and exclude peanuts and shellfish, and are gluten-free. I will provide two detailed results.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f96753a2-c541-4f1d-8a9d-59f2b2dcb9e2', 'tool_calls': [{'id': '699b7601ac56441890fe0c98fe072fa6', 'function': {'name': 'search_recipes', 'arguments': '{"addRecipeInformation": true, "cuisine": "", "diet": "gluten-free", "excludeIngredients": "peanuts, shellfish", "includeIngredients": "onion, garlic", "number": 2, "query": "chicken"}'}, 'type': 'function'}], 'token_count': {'input_tokens': 1108.0, 'output_tokens': 127.0}}, response_metadata={'document

In [99]:
let_me_see = llm_with_tools.invoke(query).tool_calls
let_me_see[0]['args']


{'addRecipeInformation': 'True',
 'cuisine': '',
 'diet': 'gluten-free',
 'excludeIngredients': 'peanuts, shellfish',
 'includeIngredients': 'chicken, onion, garlic',
 'number': 2,
 'query': 'chicken, onion, garlic'}

In [101]:
let_me_see[0]['args']['addRecipeInformation']

'True'

In [95]:
# llm_with_tools.invoke(messages)

## RAG Method to obtain the ingredients and instructions for a recipe (Working)


In [80]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://spoonacular.com/italian-kale-and-potato-soup-648155")
docs = loader.load()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())

# query = "Give me the ingredients and instructions for this recipe"
# result = db.similarity_search(query)
# result[0].page_content


llm = ChatOpenAI(model='gpt-3.5-turbo')



prompt = ChatPromptTemplate.from_template(
    """
    You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

    Your response should:
    - Start with a brief summary, if appropriate.
    - Break down complex information into clear steps or components.
    - Be concise yet thorough.

    Here is the context for your response:
    <context>
    {context}
    </context>
    
    Now, answer the following question:
    Question: {input}
    """
)



document_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()


retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "what are the ingredients and the instructions for this recipe?"})
response['answer']


'Summary:\nThe recipe is for Italian Kale and Potato Soup. The ingredients include olive oil, Italian ground turkey sausage, potatoes, onion, garlic, kale, low sodium chicken broth, dried basil, dried oregano, fennel seed, low fat buttermilk, fresh ground pepper, and shredded cheese. The instructions involve browning the sausage, sautéing onions and garlic, creating a bouquet garni with herbs, adding all ingredients except kale and buttermilk, simmering until potatoes are cooked, adding kale, stirring in buttermilk, seasoning with pepper, and serving with shredded cheese and crusty bread.\n\nIngredients:\n- 2 tsp olive oil\n- 1.25 pounds Italian ground turkey sausage\n- 1 medium potato\n- 1 medium onion\n- 3 cloves garlic\n- 1 bunch kale\n- 6 cups low sodium chicken broth\n- 2 tsp dried basil\n- 2 tsp dried oregano\n- 2 tsp fennel seed\n- 0.5 cup low fat buttermilk\n- Fresh ground pepper to taste\n- 0.25 cup shredded cheese\n\nInstructions:\n1. Heat olive oil in a large saucepan, brown

In [66]:
# Assume `documents` is the list of Document objects returned.
for doc in docs:
    description = doc.metadata.get('description', 'No description available')
    print(description)


Italian Kale and Potato Soup is a gluten free and whole 30 recipe with 6 servings. This main course has 480 calories, 22g of protein, and 33g of fat per serving. For $2.06 per serving, this recipe covers 26% of your daily requirements of vitamins and minerals. 2 people have made this recipe and would make it again. If you have oregano, ground turkey sausage, chicken broth, and a few other ingredients on hand, you can make it. It can be enjoyed any time, but it is especially good for Autumn. Not a lot of people really liked this Mediterranean dish. It is brought to you by Foodista. From preparation to the plate, this recipe takes roughly 45 minutes. All things considered, we decided this recipe deserves a spoonacular score of 80%. This score is great. Similar recipes are Italian Kale and Potato Soup, Zuppa Vegana: Italian Potato, Bean, and Kale Soup, and Zuppa Toscana {Creamy Potato & Kale Soup with Italian Sausage}.


In [75]:
for doc in docs:
    description = doc.page_content
    # Method 1: Replace multiple newlines with a single space
    cleaned_text = "\n".join(line.strip() for line in description.splitlines() if line.strip())
    print(len(cleaned_text))


7008


In [11]:
# import requests
# from dotenv import load_dotenv
# from langchain_core.tools import tool
# import os

# # Load environment variables (e.g., API key)
# load_dotenv()

# # Define the helper function to make the API request
# def search_recipes_tool(query, includeIngredients, excludeIngredients, diet, cuisine, addRecipeInformation, number):
#     # API Key and URL
#     API_KEY = os.getenv("SPOONACULAR_API_KEY")
#     BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

#     # Parameters for the search
#     params = {
#         "query": query,  # Search term
#         "includeIngredients": includeIngredients,  # Must-have ingredients
#         "excludeIngredients": excludeIngredients,  # Ingredients to exclude
#         "diet": diet,  # Dietary preference
#         "cuisine": cuisine,  # Cuisine type
#         "addRecipeInformation": addRecipeInformation,  # Include detailed recipe info
#         "number": number,  # Number of results
#         "apiKey": API_KEY  # API Key
#     }

#     # Make the API request
#     response = requests.get(BASE_URL, params=params)

#     # Check the response status and return results
#     if response.status_code == 200:
#         return response.json()  # Returning the full JSON response from Spoonacular
#     else:
#         return {"error": f"Failed to fetch recipes: {response.status_code}"}

# # Decorate the function to make it a LangChain tool
# @tool
# def search_recipes(
#     query: str, 
#     includeIngredients: str, 
#     excludeIngredients: str, 
#     diet: str, 
#     cuisine: str, 
#     addRecipeInformation: bool, 
#     number: int,
# ) -> dict:
#     """Search for recipes based on various filters and preferences.

#     Args:
#         query: Search term for the recipe (e.g., 'pasta').
#         includeIngredients: Ingredients that must be included in the recipes (comma-separated).
#         excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
#         diet: Dietary preference (e.g., 'vegetarian').
#         cuisine: Cuisine type (e.g., 'italian').
#         addRecipeInformation: Whether to include detailed recipe information (True/False).
#         number: Number of results to return.

#     Returns:
#         A dictionary containing the search results.
#     """
#     return search_recipes_tool(query, includeIngredients, excludeIngredients, diet, cuisine, addRecipeInformation, number)
